In [54]:
DATASET_NAME = "cnn_dailymail"
CONFIG = "3.0.0"

NMT_MODEL_NAME = "Helsinki-NLP/opus-mt-en-mul"
NMT_MODEL_CONFIG = {}

TRANSLATION_PREFIX = ">>nob<< "

SPLIT_PATTERN = "(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s"

MAX_INPUT_LENGTH = 1000

In [158]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from typing import List, Dict

import os
import time
import torch

import regex as re

In [3]:
dataset = load_dataset(DATASET_NAME, CONFIG)

Found cached dataset cnn_dailymail (C:/Users/navjo/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
100%|██████████| 3/3 [00:00<00:00,  6.16it/s]


In [14]:

tokenizer = AutoTokenizer.from_pretrained(NMT_MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(NMT_MODEL_NAME)


In [169]:

def prepare_input(inp: str) -> List[str]:
    sentence_splits = re.split(SPLIT_PATTERN, inp)

    #Pre-processing

    # Append >>nob<< token
    sentence_splits = [TRANSLATION_PREFIX + sentence for sentence in sentence_splits]

    return sentence_splits

def translate_batch(inp: str) -> List[str]:
    split_input = prepare_input(inp)

    input_ids = tokenizer(split_input, return_tensors="pt", padding=True).input_ids

    outputs = model.generate(input_ids)

    translated_input = ""
    for translated_sentence in outputs:
        translated_input += " " + tokenizer.decode(translated_sentence, skip_special_tokens=True)

    return translated_input

def translate_sample(inp: Dict) -> Dict:
    translated_article = translate_batch(inp["article"])
    translated_highlights = translate_batch(inp["highlights"])

    return {"article": translated_article, "highlights": translated_highlights, "id": inp["id"]}

In [170]:
res = translate_sample(dataset["train"][10])
print(res)